A pesar de contar con datos climáticos mensuales de la última década y con información turística desde 2014 hasta 2023, nos centramos específicamente en el último trimestre de 2023 para desentrañar los matices de los precios de Airbnb en esta región.

El objetivo es proporcionar una visión detallada de cómo las variables climáticas y la actividad turística se entrelazan para dar forma a las tarifas de alojamiento en Airbnb. A través de técnicas de análisis de datos, visualización y modelado predictivo, buscamos revelar patrones, correlaciones y posibles proyecciones para los precios de alojamiento en la región.

In [71]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
%run Functions.ipynb

In [72]:
data = pd.read_csv('../data/csv_rentas/Euskadi_listings.csv/listings.csv')
data.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,132068,https://www.airbnb.com/rooms/132068,20230924144519,2023-09-24,city scrape,Condo in San Sebastián · ★4.88 · 2 bedrooms · ...,"Amazing modern, clean & fully equiped flat of...",Gros es el barrio de moda de Donosti. El apart...,https://a0.muscache.com/pictures/22b9f457-1f65...,648938,https://www.airbnb.com/users/show/648938,Mikel,2011-05-31,"San Sebastian, Spain",Nací y crecí en San Sebastian. Al igual que vo...,within an hour,100%,100%,t,https://a0.muscache.com/im/users/648938/profil...,https://a0.muscache.com/im/users/648938/profil...,NaN,1,1,"['email', 'phone']",t,t,"San Sebastián, Basque Country, Spain",Donostia-San SebastiÃ¡n,GuipÃºzcoa,43.32475,-1.97335,Entire condo,Entire home/apt,4,NaN,1 bath,2.0,3.0,"[""Shampoo"", ""Stove"", ""Children\u2019s books an...",$214.00,1,63,1,4,63,63,1.6,63.0,NaN,t,2,19,40,240,2023-09-24,683,58,4,2011-06-04,2023-09-19,4.88,4.90,4.77,4.97,4.97,4.94,4.75,NaN,t,1,1,0,0,4.56
1,142760,https://www.airbnb.com/rooms/142760,20230924144519,2023-09-24,city scrape,Rental unit in Busturia · ★4.83 · 1 bedroom · ...,Beach front apartment close to San Juan de Gaz...,NaN,https://a0.muscache.com/pictures/71900ac6-da9b...,693119,https://www.airbnb.com/users/show/693119,Maitane,2011-06-12,"Madrid, Spain","World wanderer, in love with photography and c...",within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/4db6c...,https://a0.muscache.com/im/pictures/user/4db6c...,La Latina,2,4,"['email', 'phone']",t,t,NaN,Sukarrieta,Vizcaya,43.39243,-2.69474,Entire rental unit,Entire home/apt,4,NaN,1 bath,1.0,3.0,"[""Shampoo"", ""Private patio or balcony"", ""Mount...",$120.00,2,730,2,2,1125,1125,2.0,1125.0,NaN,t,30,60,90,365,2023-09-24,66,18,2,2011-08-17,2023-09-09,4.83,4.84,4.92,4.45,4.58,4.71,4.75,NaN,t,1,1,0,0,0.45
2,309802,https://www.airbnb.com/rooms/309802,20230924144519,2023-09-24,city scrape,Rental unit in San Sebastian · ★4.0 · 2 bedroo...,The apartment is located in the mezzanine of a...,In this neighborhood with a strong personality...,https://a0.muscache.com/pictures/50730314/9cfb...,1589633,https://www.airbnb.com/users/show/1589633,Emy,2012-01-10,"San Sebastian, Spain",My name is Emy and I am the owner of these apa...,within an hour,100%,97%,f,https://a0.muscache.com/im/users/1589633/profi...,https://a0.muscache.com/im/users/1589633/profi...,NaN,9,10,"['email', 'phone']",t,t,"San Sebastian, Basque Country, Spain",Donostia-San SebastiÃ¡n,GuipÃºzcoa,43.31580,-2.00598,Entire rental unit,Entire home/apt,6,NaN,2 baths,2.0,5.0,"[""Stove"", ""Oven"", ""Backyard"", ""Hair dryer"", ""C...",$190.00,3,1125,3,3,1125,1125,3.0,1125.0,NaN,t,3,28,58,290,2023-09-24,14,7,1,2014-05-06,2023-09-09,4.00,4.43,4.07

Comenzaré explorando la variación de precios durante el último trimestre de 2023, destacando posibles tendencias y outliers.

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6747 entries, 0 to 6746
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6747 non-null   int64  
 1   listing_url                                   6747 non-null   object 
 2   scrape_id                                     6747 non-null   int64  
 3   last_scraped                                  6747 non-null   object 
 4   source                                        6747 non-null   object 
 5   name                                          6747 non-null   object 
 6   description                                   6674 non-null   object 
 7   neighborhood_overview                         3634 non-null   object 
 8   picture_url                                   6747 non-null   object 
 9   host_id                                       6747 non-null   i

Primero seleccionare las columnas que están directamente relaciondas con el precio y la ubicación:

In [74]:
seleccion= ['id', 'listing_url', 'name', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 
            'room_type', 'accommodates', 'bathrooms_text','bedrooms', 'beds', 'amenities', 'price','minimum_nights',
            'minimum_nights_avg_ntm', 'maximum_nights', 'maximum_nights_avg_ntm', 'availability_365', 'first_review']
data = data[seleccion]

In [75]:
data.isna().sum()

id                           0
listing_url                  0
name                         0
neighbourhood_cleansed       0
latitude                     0
longitude                    0
property_type                0
room_type                    0
accommodates                 0
bathrooms_text              20
bedrooms                  1761
beds                        84
amenities                    0
price                        0
minimum_nights               0
minimum_nights_avg_ntm       0
maximum_nights               0
maximum_nights_avg_ntm       0
availability_365             0
first_review               808
dtype: int64

In [76]:
data.dtypes

id                          int64
listing_url                object
name                       object
neighbourhood_cleansed     object
latitude                  float64
longitude                 float64
property_type              object
room_type                  object
accommodates                int64
bathrooms_text             object
bedrooms                  float64
beds                      float64
amenities                  object
price                      object
minimum_nights              int64
minimum_nights_avg_ntm    float64
maximum_nights              int64
maximum_nights_avg_ntm    float64
availability_365            int64
first_review               object
dtype: object

Manejo el formato del precio asegurándome que la columna 'price' tenga un formato numérico adecuado:

In [77]:
'''
Primero elimino el símbolo '$' y lo convierto a tipo de dato numérico. Luego, aplico la tasa de cambio de 
dólares a euros. Actualmente 1 euror = 1.09$
'''

data['price']=data['price'].replace('[\$]', '', regex=True).replace(',', '', regex=True).astype(float)
conversion = 1.09
data['price']=data['price']*conversion
data.head()

,id,listing_url,name,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,minimum_nights_avg_ntm,maximum_nights,maximum_nights_avg_ntm,availability_365,first_review
0,132068,https://www.airbnb.com/rooms/132068,Condo in San Sebastián · ★4.88 · 2 bedrooms · ...,Donostia-San SebastiÃ¡n,43.32475,-1.97335,Entire condo,Entire home/apt,4,1 bath,2.0,3.0,"[""Shampoo"", ""Stove"", ""Children\u2019s books an...",233.26,1,1.6,63,63.0,240,2011-06-04
1,142760,https://www.airbnb.com/rooms/142760,Rental unit in Busturia · ★4.83 · 1 bedroom · ...,Sukarrieta,43.39243,-2.69474,Entire rental unit,Entire home/apt,4,1 bath,1.0,3.0,"[""Shampoo"", ""Private patio or balcony"", ""Mount...",130.80,2,2.0,730,1125.0,365,2011-08-17
2,309802,https://www.airbnb.com/rooms/309802,Rental unit in San Sebastian · ★4.0 · 2 bedroo...,Donostia-San SebastiÃ¡n,43.31580,-2.00598,Entire rental unit,Entire home/apt,6,2 baths,2.0,5.0,"[""Stove"", ""Oven"", ""Backyard"", ""Hair dryer"", ""C...",207.10,3,3.0,1125,1125.0,290,2014-05-06
3,309813,https://www.airbnb.com/rooms/309813,Rental unit in San Sebastian · ★4.67 · 3 bedro...,Donostia-San SebastiÃ¡n,43.31681,-1.98611,Entire rental unit,Entire home/apt,8,2 baths,3.0,5.0,"[""32\"" TV"", ""Pack \u2019n play/Travel crib"", ""...",196.20,31,31.0,124,124.0,323,2012-03-15
4,309828,https://www.airbnb.com/rooms/309828,Rental unit in San Sebastian · ★5.0 · 3 bedroo...,Donostia-San SebastiÃ¡n,43.31471,-1.98588,Entire rental unit,Entire home/apt,8,2 baths,3.0,5.0,"[""Pack \u2019n play/Travel crib"", ""Stove"", ""Ov...",299.75,31,31.0,124,124.0,325,2014-01-04


In [78]:
data['bathrooms'] = data['bathrooms_text'].str.extract('(\d+)').astype(float)

In [79]:
data.head()

,id,listing_url,name,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,minimum_nights_avg_ntm,maximum_nights,maximum_nights_avg_ntm,availability_365,first_review,bathrooms
0,132068,https://www.airbnb.com/rooms/132068,Condo in San Sebastián · ★4.88 · 2 bedrooms · ...,Donostia-San SebastiÃ¡n,43.32475,-1.97335,Entire condo,Entire home/apt,4,1 bath,2.0,3.0,"[""Shampoo"", ""Stove"", ""Children\u2019s books an...",233.26,1,1.6,63,63.0,240,2011-06-04,1.0
1,142760,https://www.airbnb.com/rooms/142760,Rental unit in Busturia · ★4.83 · 1 bedroom · ...,Sukarrieta,43.39243,-2.69474,Entire rental unit,Entire home/apt,4,1 bath,1.0,3.0,"[""Shampoo"", ""Private patio or balcony"", ""Mount...",130.80,2,2.0,730,1125.0,365,2011-08-17,1.0
2,309802,https://www.airbnb.com/rooms/309802,Rental unit in San Sebastian · ★4.0 · 2 bedroo...,Donostia-San SebastiÃ¡n,43.31580,-2.00598,Entire rental unit,Entire home/apt,6,2 baths,2.0,5.0,"[""Stove"", ""Oven"", ""Backyard"", ""Hair dryer"", ""C...",207.10,3,3.0,1125,1125.0,290,2014-05-06,2.0
3,309813,https://www.airbnb.com/rooms/309813,Rental unit in San Sebastian · ★4.67 · 3 bedro...,Donostia-San SebastiÃ¡n,43.31681,-1.98611,Entire rental unit,Entire home/apt,8,2 baths,3.0,5.0,"[""32\"" TV"", ""Pack \u2019n play/Travel crib"", ""...",196.20,31,31.0,124,124.0,323,2012-03-15,2.0
4,309828,https://www.airbnb.com/rooms/309828,Rental unit in San Sebastian · ★5.0 · 3 bedroo...,Donostia-San SebastiÃ¡n,43.31471,-1.98588,Entire rental unit,Entire home/apt,8,2 baths,3.0,5.0,"[""Pack \u2019n play/Travel crib"", ""Stove"", ""Ov...",299.75,31,31.0,124,124.0,325,2014-01-04,2.0
